In [84]:
import pandas as pd

In [85]:
df = pd.read_csv("spotify_millsongdata.csv")

In [86]:
df.head(5)

,artist,song,link,text
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \r\nA..."
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \r\nTouch me gen..."
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go \r\nWhy I had...
3,ABBA,Bang,/a/abba/bang_20598415.html,Making somebody happy is a question of give an...
4,ABBA,Bang-A-Boomerang,/a/abba/bang+a+boomerang_20002668.html,Making somebody happy is a question of give an...


In [87]:
df.tail(5)

,artist,song,link,text
57645,Ziggy Marley,Good Old Days,/z/ziggy+marley/good+old+days_10198588.html,Irie days come on play \r\nLet the angels fly...
57646,Ziggy Marley,Hand To Mouth,/z/ziggy+marley/hand+to+mouth_20531167.html,Power to the workers \r\nMore power \r\nPowe...
57647,Zwan,Come With Me,/z/zwan/come+with+me_20148981.html,all you need \r\nis something i'll believe \...
57648,Zwan,Desire,/z/zwan/desire_20148986.html,northern star \r\nam i frightened \r\nwhere ...
57649,Zwan,Heartsong,/z/zwan/heartsong_20148991.html,come in \r\nmake yourself at home \r\ni'm a ...


In [88]:
df.shape

(57650, 4)

In [89]:
df.isnull().sum()

artist    0
song      0
link      0
text      0
dtype: int64

In [90]:
df =df.sample(5000).drop('link', axis=1).reset_index(drop=True)

In [91]:
df.head(10)

,artist,song,text
0,Nitty Gritty Dirt Band,Propinquity,I've known for a long time \r\nThe kind of gi...
1,Weird Al Yankovic,The Check's In The Mail,"Well, hey how ya doin' have a seat, have a dri..."
2,George Harrison,Baby Don't Run Away,Baby don't run away from me \r\nBaby don't go...
3,Tim McGraw,You Got The Wrong Man,Your eyes say that you've been hurt before \r...
4,Steve Miller Band,Rock Love,Do do do do doot \r\nDo do do do doot \r\nDo...
5,John Prine,If She Were You,"Well, everytime I try to love another \r\nI n..."
6,Kate Bush,Sister And Brother,"Sister, I will stand by you \r\nSister, I bel..."
7,Allman Brothers Band,Famous Last Words,"Famous last words, I'll be right back in a lit..."
8,Bette Midler,The Glory Of Love,"""This is a song I've been singing for a long t..."
9,Ray Charles,It Was A Very Good Year,When I was seventeen \r\nIt was a very good y...


In [92]:
df['text'][0]

"I've known for a long time  \r\nThe kind of girl you are  \r\nAnd a smile that covers teardrops  \r\nThe way your head yields to your heart  \r\nOf things you've kept inside  \r\nThat most girls couldn't bear  \r\nWell, I've known you for a long time  \r\nBut I've just begun to care  \r\nYes, I've known of all the heartaches  \r\nAnd I've known of all the pain  \r\nI've seen you when the the sun shines  \r\nAnd I've seen you when it rains  \r\nI've seen you make a look of love  \r\nFrom just an icy stare  \r\nWell, I've known you for a long time  \r\nBut I've just begun to care  \r\nI know I've been blind  \r\nTo not have loved you all this time  \r\nTo not have loved you all this time  \r\nBut the image of you wasn't clear  \r\nI guess I've been standing too near  \r\nIt's taken me a while  \r\nBut I have finally found  \r\nWhat you are to me  \r\nAnd that's what really counts  \r\nAnd what you are to me  \r\nIs something we can share  \r\nWell, I've known you for a long time  \r\nBu

In [93]:
df.shape

(5000, 3)

In [94]:
df['text'] = df['text'].str.lower().replace(r'^\w\s', ' ').replace(r'\n', ' ', regex = True)

In [95]:
import nltk
from nltk.stem.porter import PorterStemmer

In [96]:
stemmer = PorterStemmer()

In [97]:
def tokenization(txt):
    tokens = nltk.word_tokenize(txt)
    stemming = [stemmer.stem(w) for w in tokens]
    return " ".join(stemming)

In [98]:
df['text'] = df['text'].apply(lambda x: tokenization(x))

In [99]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [100]:
tfidvector = TfidfVectorizer(analyzer='word',stop_words='english')
matrix = tfidvector.fit_transform(df['text'])
similarity = cosine_similarity(matrix)

In [101]:
similarity[0]

array([1.        , 0.0197688 , 0.05517578, ..., 0.02109082, 0.0087496 ,
       0.01247753])

In [102]:
df[df['song'] == 'Giving Up']

,artist,song,text
2566,Ingrid Michaelson,Giving Up,what if we stop have a ball ? what if the pain...


In [103]:
def recommendation(song_df):
    idx = df[df['song'] == song_df].index[0]
    distances = sorted(list(enumerate(similarity[idx])),reverse=True,key=lambda x:x[1])
    
    songs = []
    for m_id in distances[1:21]:
        songs.append(df.iloc[m_id[0]].song)
        
    return songs

In [104]:
recommendation('Giving Up')

['September Grass',
 "And The Grass Won't Pay No Mind",
 "And The Grass Won't Pay No Mind",
 'When Somebody Loves You',
 'She Closed Her Eyes',
 'Anything But Strong',
 '(all I Can Do Is) Dream You',
 'Why Does It Feel So Good',
 'My Little Girl',
 'Kimberly',
 "I'm Always Chasing Rainbows",
 'Blue',
 'Affirmation',
 'Let Go',
 'Lonely People',
 'Home',
 "Don't Forget About Us",
 'Driftwood',
 'I Do What I Love',
 'Heart Of Stone']

In [105]:
import pickle
pickle.dump(similarity,open('similarity.pkl','wb'))
pickle.dump(df,open('df.pkl','wb'))